## 1. Isolation Forest with contamination = 0.01

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='darkgrid')

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/kaggle/input/private-fraud-detection-dataset/transactions_train.csv', index_col=0)
df.head()

,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
step,,,,,,,,,
1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6351193 entries, 1 to 699
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   type            object 
 1   amount          float64
 2   nameOrig        object 
 3   oldbalanceOrig  float64
 4   newbalanceOrig  float64
 5   nameDest        object 
 6   oldbalanceDest  float64
 7   newbalanceDest  float64
 8   isFraud         int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 484.6+ MB


In [5]:
df.isFraud.value_counts()

0    6343476
1       7717
Name: isFraud, dtype: int64

- A very highly imbalanced dataset.
- We cannot use sampling techniques. It will take up a lot of resources. 
- 9 features. We do not have to worry about dimensionality problem. 
- The primary issue would be the highly imbalances dataset,

### ISOLATION FOREST

In [6]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()
categorical_columns = ['type', 'nameOrig', 'nameDest']
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

In [10]:
# Define the features and target variable
X = df.drop('isFraud', axis=1)
y = df['isFraud']

In [8]:
#The contamination hyperparameter controls the proportion of anomalies that we expect in the dataset. 
# We will adjust this value using hyperparameter optimization.

isolation_forest = IsolationForest(contamination=0.01, random_state=42)

In [11]:
# Fit the model on your data
isolation_forest.fit(X)

IsolationForest(contamination=0.01, random_state=42)

In [12]:
# Predict anomalies (outliers)
predictions = isolation_forest.predict(X)

In [13]:
# Evaluate the model's performance (e.g., precision, recall, F1-score)
from sklearn.metrics import classification_report
print(classification_report(y, [1 if pred == -1 else 0 for pred in predictions]))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99   6343476
           1       0.01      0.05      0.01      7717

    accuracy                           0.99   6351193
   macro avg       0.50      0.52      0.50   6351193
weighted avg       1.00      0.99      0.99   6351193

